## 判断质数
思路：小于1w的质数判断正确，大于1w全都是质数；另外工具是费马那个，有几个数会判断错.正常情况应该使用

In [ ]:
def primes_below_N(N):
    if N < 2:
        return []
    
    # 初始化一个布尔数组，"True"表示对应的数是质数
    is_prime = [True] * N
    is_prime[0], is_prime[1] = False, False  # 0和1不是质数
    
    for i in range(2, int(N ** 0.5) + 1):
        if is_prime[i]:
            # 将i的倍数标记为非质数
            for j in range(i*i, N, i):
                is_prime[j] = False
                
    # 返回所有标记为True的索引，即质数
    return [i for i in range(2, N) if is_prime[i]]

# 示例调用
N = 300000
primes_1w = primes_below_N(10000)

In [ ]:
def detect_prime_clear(n):
    # 精确的检测是否是质数，在小于1w的情况下精确，否则都输出是质数
    if n < 10000:
        if n in primes_1w:
            return True
        else:
            return False
    else:
        return True

In [14]:
def power_mod(base, exponent, modulus):
    """
    快速计算 (base^exponent) % modulus 的值。
    """
    result = 1
    base = base % modulus
    while exponent > 0:
        if exponent % 2 == 1:  # 如果当前指数是奇数
            result = (result * base) % modulus
        exponent = exponent // 2  # 使用整除代替位操作以提高可读性
        base = (base * base) % modulus
    return result

def fermat_primality_test_fixed_bases(n):
    """
    使用固定的基底集合进行费马小定理素性测试。
    n: 需要测试的数。
    返回True表示n可能是质数，False表示n不是质数。
    """
    if n <= 1 or n == 4:  # 显然不是质数的情况
        return False
    if n <= 3:  # 小于等于3的正整数都是质数
        return True
    
    bases = [2, 3]  # 固定的基底集合
    for a in bases:
        if power_mod(a, n - 1, n) != 1:
            return False
    return True

# 示例调用
n = 1165
print(f"数字{n} {'可能是质数' if fermat_primality_test_fixed_bases(n) else '不是质数'}")

数字1165 不是质数


In [15]:
def test_carmichael_numbers():
    carmichael_numbers = list(set(range(10000))- set(primes_1w))
    for n in carmichael_numbers:
        result = fermat_primality_test_fixed_bases(n)
        if  result: 
            print(f"数字{n} {'可能是质数' if result else '不是质数'}")

# 调用上面定义的fermat_primality_test_fixed_bases函数进行测试
test_carmichael_numbers()

数字1105 可能是质数
数字1729 可能是质数
数字2465 可能是质数
数字2701 可能是质数
数字2821 可能是质数
数字6601 可能是质数
数字8911 可能是质数


## 汇率计算
正常情况是获得实时汇率，但是没有网的话实时汇率全都是nan；可以获取基础汇率。设置一些情况允许使用估算汇率

In [128]:
import random

# 设置固定的随机种子以保证结果可复现
random.seed(42)

# 假设的货币及其之间的汇率（使用代指货币A、B、C、D、E）
exchange_rates = {
    'A': {'B': 0.85, 'C': 110.25, 'D': 0.75},
    'B': {'A': 1.18, 'C': 130.00, 'D': 0.88},
    'C': {'A': 0.0091, 'B': 0.0077, 'D': 0.0068},
    'D': {'A': 1.34, 'B': 1.14, 'C': 147.06}
    # 添加更多货币和汇率...
}

def get_base_exchange_rate(from_currency, to_currency):
    """
    获取两个货币之间的汇率。
    
    参数:
        from_currency (str): 来源货币代码
        to_currency (str): 目标货币代码
        
    返回:
        float: 汇率
    """
    base_rate = exchange_rates[from_currency][to_currency]
    return base_rate

def get_realtime_exchange_rate(from_currency, to_currency):
    """
    获取两个货币之间的实时汇率。
    
    参数:
        from_currency (str): 来源货币代码
        to_currency (str): 目标货币代码
        
    返回:
        float: 实时汇率
    """
    base_rate = exchange_rates[from_currency][to_currency]
    fluctuation = random.uniform(-0.05, 0.05) * base_rate  # 随机波动范围为±5%
    real_time_rate = base_rate + fluctuation
    return real_time_rate

print(f"查询到的基础汇率: {get_base_exchange_rate('A', 'B'):.4f}")
print(f"查询到的实时汇率: {get_realtime_exchange_rate('A', 'B'):.4f}")

查询到的基础汇率: 0.8500
查询到的实时汇率: 0.8619


## 文件操作
文件检索可能出问题的地方：加密文件；权限不足；文件格式不支持；文件过大无法移动/复制

对应的解决办法：文件解密；提高权限；文件格式转换；文件切分与合并

文件移动是简单的

In [4]:
from pydantic import BaseModel, Field
from datetime import datetime
import shutil
import os

class FileSim(BaseModel):
    file_name: str
    size: int = Field(..., description="File size in bytes")
    created_at: datetime = Field(default_factory=datetime.now)
    encrypted: bool = False
    absolute_path: str
    
    def copy_file(self, new_path: str):
        """Simulate copying a file to a new path."""
        print(f"Copying {self.file_name} to {new_path}")
        # In real application, you would use shutil.copy or similar here.
        new_instance = self.model_copy(update={'absolute_path': new_path})
        return new_instance
    
    def move_file(self, new_path: str):
        """Simulate moving a file to a new path."""
        print(f"Moving {self.file_name} to {new_path}")
        # Update the absolute path of this file instance
        updated_fields = {'absolute_path': new_path}
        return self.model_copy(update=updated_fields)
    
    def rename_file(self, new_name: str):
        """Rename the file."""
        print(f"Renaming {self.file_name} to {new_name}")
        return self.model_copy(update={'file_name': new_name})
    
    def split_file(self, split_size: int):
        """Simulate splitting the file into multiple parts."""
        print(f"Splitting {self.file_name} into parts of {split_size} bytes.")
        # This is just a simulation; actual implementation may vary.
        pass
    
    def merge_files(self, files_to_merge):
        """Simulate merging multiple files into one."""
        print(f"Merging {self.file_name} with {files_to_merge}")
        # This is just a simulation; actual implementation may vary.
        pass

# Example usage:
if __name__ == "__main__":
    file = FileSim(file_name="example.txt", size=1024, absolute_path="/user/home/example.txt")
    file_moved = file.move_file("/user/documents/")
    print(file_moved)
    file_renamed = file.rename_file("new_example.txt")
    file_copied = file.copy_file("/backup/example.txt")
    file.split_file(512)
    file.merge_files(["/path/to/file1", "/path/to/file2"])

Moving example.txt to /user/documents/
file_name='example.txt' size=1024 created_at=datetime.datetime(2025, 2, 16, 12, 10, 6, 635952) encrypted=False absolute_path='/user/documents/'
Renaming example.txt to new_example.txt
Copying example.txt to /backup/example.txt
Splitting example.txt into parts of 512 bytes.
Merging example.txt with ['/path/to/file1', '/path/to/file2']
